In [2]:
from z3 import *

from itertools import combinations
from z3 import *
from utils import *
import math

In [3]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

In [4]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

In [5]:
def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name+'1'), at_least_k_seq(bool_vars, k, name))

In [8]:
'''
def num_packs_cour(numCour, v):
   # Returns the number of packs that numCour holds
   return Sum(v[numCour])

#Tutto ciò che segue è un po' inutile. Deve ritornare una variabile!


def num_packs_path_cour(numCour, v, pred):
   #Return the number of packs in a path done by numCour
   
   n = len(v[numCour]) #numero pacchi
   i = Int('i')
   A = Array('A', IntSort(), BoolSort())
   for k in range(n):
      Store(A,k, v[numCour][k])

   mini_s = Solver() 
   mini_s.add(i>=0)
   mini_s.add(i<n)
   mini_s.add(A[i]==True)

   if mini_s.check()==unsat: #If numCour does not hold any pack return 0
      return 0
   
   mini_mod = mini_s.model()

   one_pack = mini_mod.evaluate(i).as_long() #One random pack of the courier numCour
   path_list=[] #List of packs in the current observed path
   at_the_starting_point = False
   at_the_ending_point = False

   one_new_pack = one_pack
   while (not at_the_starting_point) and (one_new_pack not in path_list): #If not at the deposit AND the current pack is not already been visted
      
      path_list.append(one_new_pack) #add the new pack to the list of packs

      A = Array('A', IntSort(), BoolSort())
      i = Int('i')
      mini_s = Solver() #Will find the predecessor of one_new_pack
      for k in range(n):
         Store(A, k, pred[one_new_pack][k])
      mini_s.add(i>=0)
      mini_s.add(i<n)
      mini_s.add(A[i]==True)
      mini_s.check()
      mini_mod = mini_s.model()
      one_new_pack = mini_mod.evaluate(i).as_long()
      
      if one_new_pack == n:
         at_the_starting_point=True #If we are at the deposit

   one_new_pack = one_pack
   path_list.remove(one_pack) #remove one_pack so the following while can start!
   while (not at_the_ending_point) and (one_new_pack not in path_list):

      path_list.append(one_new_pack)

      A = Array('A', IntSort(), BoolSort())
      i = Int('i')

      mini_s = Solver()
      for k in range(n):
         Store(A,k, pred[k][one_new_pack])
      mini_s.add(i>=0)
      mini_s.add(i<n)
      mini_s.add(A[i]==True)
      mini_s.check()
      mini_mod = mini_s.model()
      one_new_pack = mini_mod.evaluate(i).as_long()
      
      if one_new_pack == n:
         at_the_ending_point=True
      
   return len(path_list)
   

#Ci riprovo :'(
#Ora non ne ho la testa per farlo ma credo andrebbe fatto fuori dalla funzione il primo pacco di ogni corriere perchè quel new_constraints
#deve essere add al solver (lega dinamicamente one_pack a un pacco). 

def num_packs_path_cour(numCour, v, pred):
   #Return the number of packs in a path done by numCour
   one_pack = Int('one_pack')
   new_constraints = [Or([And([one_pack==k , v[numCour][k]==True]) for k in range(10)])]
   
   '''


In [22]:
'''#PROVE INUTILI:

n = Int('n')
lista_BLA = [Bool(f"BLA{j}")for j in range(10)]

new_constraint = Or([And([n==k , lista_BLA[k]==True]) for k in range(10)])


aaa = Solver()
aaa.add(Sum(lista_BLA)==2)
aaa.add(lista_BLA[7]==True)
aaa.add(new_constraint)
aaa.check()

aaa_mod=aaa.model()
print([aaa_mod.evaluate(lista_BLA[i]) for i in range(10)], aaa_mod.evaluate(n))


A = BoolVector('A', 10)
for i in lista_BLA:
   A=lista_BLA


def prova(n):
    aaa=Solver()
    aaa.add(n>3)
    aaa.check()
    aaa_mod=aaa.model()
    print(aaa_mod.evaluate(n).as_long())
    return aaa_mod.evaluate(n).as_long()

def prova_2(x, y):
    f = Function('F', IntSort(), IntSort())
    return f(x)!=y

x = Bool('x')
y = Bool('y')
bbb = Solver()
bbb.add(prova_2(x,y))
bbb.check()
bbb_mod=bbb.model()
print(bbb_mod.evaluate(x), bbb_mod.evaluate(y), bbb_mod.evaluate(prova_2(x,y)))
'''

[True, False, False, False, False, False, False, True, False, False] 0
True True True


In [31]:
'''n packs
   m couriers
   D matix int(n+1,m+1)
   sz array of item's sizes
   cpt array of couriers' capacities
'''
def mcp(n, m, D, sz, cpt):
   
   '''
      pred matrix  
   '''
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+1)]for i in range(n+1)]

   #create Solver
   solv= Solver()

   # The deposit has at most n predecessor and it is predecessor of at most n items
   col_n = []
   for j in range(n):
      col_n += [pred[j][n]]
   solv.add(at_most_k_seq(pred[n],m, f"PR_{n}")) #pred row (predecessors of the deposit -- last item of each courier)
   solv.add(at_most_k_seq(col_n,m, f"PC_{n}")) #pred col (items that have deposit as predecessor -- first items of each courier)
   print(solv.check())

   #Each item has exactly one predecessor and each item is predecessor of exactly one other item
   #i=0 to i=n-1 constraint At most one
   #j=0 to j=n-1 constraint At most one
   for i in range(n):
      col_i = []
      for j in range(n+1):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

   
   '''
      vehicle matrix 
      col = packs
      rows = courriers 
      each rows represent the list of packs foreach courrier
   ''' 
   #create a [m x n] matrix v. v[i][j]==True if the i-th courier takes the j-th item
   v=[[Bool(f"vehicle({i})_{j}")for j in range(n)]for i in range(m)]

   #each pack have to be carried by exactly one courrier
   for j in range(n):
      solv.add(exactly_one_seq([v[i][j] for i in range(m)], f"v_{j}"))
   
   #FUNZIONA. WOW :D
   solv.add(And( [Implies( And([pred[i][n], pred[j][n]]) , And([ Not(And(v[k][i], v[k][j])) for k in range(m)])) for i in range(n) for j in range(n) if i<j])) #Se due items hanno come predecessore il deposito allora devono essere presi da due veicoli diversi

   
   '''
      no_loops 
   '''
   no_loops = IntVector('no_loops', n)
   solv.add(And([Implies(pred[i][j], no_loops[i]>no_loops[j]) for i in range(n) for j in range(n)]))
   solv.add(no_loops[0]==1)



   solv.check()
   mod = solv.model()
   #print(mod)
   #print(mod.evaluate(pred[0][0]))
   
   return [(i, j) for j in range(n) for i in range(m) if mod.evaluate(v[i][j])], [(i,j) for j in range(n+1) for i in range(n+1) if mod.evaluate(pred[i][j])]

print(mcp(10,3,0,0,0))



sat
([(1, 0), (2, 1), (2, 2), (2, 3), (0, 4), (0, 5), (2, 6), (2, 7), (1, 8), (0, 9)], [(1, 0), (2, 1), (5, 2), (9, 3), (10, 4), (6, 5), (7, 6), (8, 7), (10, 8), (10, 9), (0, 10), (3, 10), (4, 10)])


10


In [ ]:
def print_items_for_each_courier(couriers_items, items_predecessors):
    
    # Crea un dizionario per memorizzare gli articoli per ogni corriere
    couriers_dict = {}
    for courier, item in couriers_items:
        if courier not in couriers_dict:
            couriers_dict[courier] = []
        couriers_dict[courier].append(item)

    # Crea un dizionario per memorizzare i predecessori di ogni articolo
    predecessors_dict = {item: predecessor for predecessor, item in items_predecessors}

    # Per ogni corriere, segui i predecessori per ottenere l'ordine degli articoli
for courier, items in couriers_dict.items():
    print(f"Corriere {courier}: ", end="")
    for item in items:
        # Crea un set per memorizzare gli articoli che abbiamo già visto
        seen_items = set()
        # Segui i predecessori fino a quando non raggiungi l'articolo senza predecessore
        while item in predecessors_dict and item not in seen_items:
            print(item, "-> ", end="")
            seen_items.add(item)
            item = predecessors_dict[item]
        print(item)  # Stampa l'ultimo articolo

# Usa la funzione
print_items_for_each_courier([(1, 0), (2, 1), (2, 2), (2, 3), (0, 4), (0, 5), (2, 6), (2, 7), (1, 8), (0, 9)], [(1, 0), (2, 1), (5, 2), (9, 3), (10, 4), (6, 5), (7, 6), (8, 7), (10, 8), (10, 9), (0, 10), (3, 10), (4, 10)])